In [2]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date, timedelta, datetime
import numpy as np

######## Select ticker, initial date, final date, initial amount, periodic amount to invest and periodicity in week days
ticker = "AAPL.MX"
initial_date = date(2023,1,1)
final_date = date.today()
df = yf.download(ticker,initial_date,final_date, progress=False)
df.columns = df.columns.droplevel(1)
##############################


df['SMA_20'] = df['Close'].rolling(window=20).mean()
df['STD_20'] = df['Close'].rolling(window=20).std()
df['BB_upper'] = df['SMA_20'] + 2 * df['STD_20']
df['BB_lower'] = df['SMA_20'] - 2 * df['STD_20']

df['SMA_200'] = df['Close'].rolling(window=200).mean()

df['Volume_MA20'] = df['Volume'].rolling(window=20).mean()

df['Breakout_Up'] = (df['Close'] > df['BB_upper']) & (df['Volume'] > df['Volume_MA20'])
df['Breakout_Down'] = (df['Close'] < df['BB_lower']) & (df['Volume'] > df['Volume_MA20'])

df['Volume_Color'] = df.apply(lambda row: 'green' if row['Close'] >= row['Open'] else 'red', axis=1)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05, subplot_titles=('OHLC con Bandas de Bollinger y SMA 200', 'Volumen'),
                    row_width=[0.2, 0.7])  # Volumen más pequeño en la segunda fila

fig.add_trace(go.Candlestick(
    x=df.index, open=df["Open"], high=df["High"], low=df["Low"], close=df["Close"],
    name="OHLC"
), row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['BB_upper'], line=dict(color='blue', width=1), name='Upper Band'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], line=dict(color='orange', width=1), name='20-Day MA'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['BB_lower'], line=dict(color='blue', width=1), name='Lower Band'), row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['SMA_200'], line=dict(color='purple', width=2, dash='dot'), name='SMA 200'), row=1, col=1)

'''
for i in df.index:
    if df.loc[i, 'Breakout_Up']:  # Ruptura alcista
        #fig.add_trace(go.Scatter(x=[i, i], y=[df['Low'].min(), df['High'].max()], mode="lines",
                                 line=dict(color="green", width=1), name="Ruptura Alcista"),
                      row=1, col=1)
    if df.loc[i, 'Breakout_Down']:  # Ruptura bajista
        #fig.add_trace(go.Scatter(x=[i, i], y=[df['Low'].min(), df['High'].max()], mode="lines",
                                 line=dict(color="red", width=1), name="Ruptura Bajista"),
                      row=1, col=1)
'''

for i in range(len(df)):
    fig.add_trace(go.Bar(
        x=[df.index[i]], y=[df['Volume'].iloc[i]],
        marker=dict(color=df['Volume_Color'].iloc[i]),
        showlegend=False  # No repetir leyenda en cada barra
    ), row=2, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['Volume_MA20'], line=dict(color='orange', width=2),
                         name='Volume MA 20'), row=2, col=1)

fig.update_layout(title=f"Análisis de {ticker} con Bandas de Bollinger, SMA 200 y Volumen",
                  xaxis_rangeslider_visible=False,
                  showlegend=True)

fig.show()
# RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = calculate_rsi(df['Close'], window=14)

# MACD
def calculate_macd(data, fast_length=12, slow_length=26, signal_smooth=9):
    ema_fast = data.ewm(span=fast_length, adjust=False).mean()
    ema_slow = data.ewm(span=slow_length, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signal_smooth, adjust=False).mean()
    histogram = macd - signal
    return macd, signal, histogram

df['MACD'], df['Signal'], df['MACD_Histogram'] = calculate_macd(df['Close'])

# ADX
def calculate_adx(df, window=14):
    high = df['High']
    low = df['Low']
    close = df['Close']

    # Calcular los movimientos de alta y baja
    up_move = high.diff()
    down_move = low.diff()

    # Calcular los valores de +DM y -DM
    plus_dm = pd.Series(np.where((up_move > down_move) & (up_move > 0), up_move, 0), index=df.index)
    minus_dm = pd.Series(np.where((down_move > up_move) & (down_move > 0), down_move, 0), index=df.index)

    # Calcular el True Range (TR) y el Average True Range (ATR)
    tr = pd.concat([high - low, abs(high - close.shift()), abs(low - close.shift())], axis=1).max(axis=1)
    atr = tr.rolling(window=window).mean()

    # Calcular el +DI y -DI
    plus_di = 100 * (plus_dm.rolling(window=window).sum() / atr)
    minus_di = 100 * (minus_dm.rolling(window=window).sum() / atr)

    # Calcular el ADX
    dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
    adx = dx.rolling(window=window).mean()

    return adx, plus_di, minus_di

df['ADX'], df['+DI'], df['-DI'] = calculate_adx(df)


def calcular_cv(df):

    if df.empty:
        print("Error: No se encontraron datos para el ticker especificado.")
        return None

    # Calcular desviación estándar de los precios de cierre
    std_dev = np.std(df['Close'])

    # Calcular precio promedio
    mean_price = np.mean(df['Close'])

    # Calcular Coeficiente de Variación (CV)
    cv = std_dev / mean_price if mean_price != 0 else None

    return cv


# Mostrar resultados
print(df[['Close', 'RSI', 'MACD', 'Signal', 'ADX', '+DI', '-DI']].tail())

print ("Coeficiente de Variación = ", calcular_cv(df))

Price             Close        RSI       MACD     Signal        ADX  \
Date                                                                  
2025-01-16  4767.240234  23.315731 -26.694225  18.908999  42.406293   
2025-01-17  4768.990234  25.203601 -36.301420   7.866916  48.070899   
2025-01-20  4840.000000  28.755336 -37.750136  -1.256495  51.311245   
2025-01-21  4579.830078  20.674286 -59.209264 -12.847049  51.455628   
2025-01-22  4577.500000  25.532579 -75.533098 -25.384258  53.353180   

Price             +DI         -DI  
Date                               
2025-01-16  29.778023  183.662880  
2025-01-17  22.187470  192.828988  
2025-01-20  66.765423  181.725129  
2025-01-21  55.124093   87.265597  
2025-01-22  62.460495  129.638840  
Coeficiente de Variación =  0.20104286627641527


### 1. RSI (Relative Strength Index)

El RSI mide la velocidad y el cambio de los movimientos de precios, con valores entre 0 y 100. Los valores por encima de 70 indican sobrecompra y los valores por debajo de 30 indican sobreventa.

### 2. MACD (Moving Average Convergence Divergence)

El MACD es un indicador de tendencia que muestra la relación entre dos medias móviles exponenciales (EMA).

### 3. ADX (Average Directional Index)

El ADX mide la fuerza de la tendencia. Un valor por encima de 25 indica una tendencia fuerte, mientras que un valor por debajo de 20 indica una tendencia débil o sin tendencia.

### 4. Coeficiente de Variación (CV)

La volatilidad mide cuánto varía el precio de un activo en un período de tiempo. A mayor volatilidad, mayor incertidumbre y riesgo. Existen varios métodos para cuantificarla: Coeficiente de Variación (CV)

Mide la volatilidad relativa respecto al precio promedio.

📌 Fórmula:

$CV = \frac{\sigma}{\bar{P}}$

Donde:
	•	 \sigma  = Desviación estándar.
	•	 \bar{P}  = Precio promedio.

💡 Interpretación:
	•	CV alto = Activo muy volátil respecto a su precio.
	•	CV bajo = Activo estable.

### 5. Índice de Volatilidad (VIX)

El VIX mide la volatilidad esperada en el S&P 500 basado en opciones.


	•	VIX alto (>30) = Alta incertidumbre, miedo en el mercado.
	•	VIX bajo (<20) = Baja incertidumbre, estabilidad.

Se puede consultar el VIX en Yahoo Finance con el ticker ^VIX.

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

n = 5  # Predicción a n días

# 🔹 1️⃣ Descargar datos usando la celda anterior

# 🔹 2️⃣ Seleccionar las columnas necesarias
data = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

# 🔹 3️⃣ Crear características adicionales
data['SMA_5'] = data['Open'].rolling(window=5).mean()
data['SMA_10'] = data['Open'].rolling(window=10).mean()

# 📌 Bandas de Bollinger (20 días, 2 desviaciones estándar)
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['StdDev_20'] = data['Close'].rolling(window=20).std()
data['BB_upper'] = data['SMA_20'] + 2 * data['StdDev_20']
data['BB_lower'] = data['SMA_20'] - 2 * data['StdDev_20']
data['BB_width'] = data['BB_upper'] - data['BB_lower']  # Ancho de las bandas

# Eliminar filas con valores NaN generados por los cálculos
data.dropna(inplace=True)

# 🔹 4️⃣ Definir X (features) y y (target)
X = data[['Open', 'Volume', 'SMA_5', 'SMA_10', 'BB_upper', 'BB_lower', 'BB_width']]

# Desplazar la variable objetivo 'Open' para predecir a n días (por ejemplo, 3 días)
y = data['Open'].shift(-n)

# Eliminar filas con valores NaN generados por el desplazamiento de la variable objetivo
X = X[:-n]
y = y.dropna()

# 🔹 5️⃣ Normalizar los datos
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)

# Normalizar la variable objetivo (también la escalamos)
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# 🔹 6️⃣ Preparar los datos para LSTM
def create_lstm_dataset(data, look_back=60):
    X_data, y_data = [], []
    for i in range(look_back, len(data)):
        X_data.append(data[i-look_back:i, :])  # Aseguramos que X_data tenga la forma correcta
        y_data.append(data[i, 0])  # Usamos el precio de apertura como objetivo
    return np.array(X_data), np.array(y_data)

look_back = 60  # Número de días previos a la predicción
X_lstm, y_lstm = create_lstm_dataset(X_scaled, look_back)

# 🔹 7️⃣ Redimensionar para LSTM: (muestras, time_steps, características)
X_lstm = X_lstm.reshape(X_lstm.shape[0], X_lstm.shape[1], X_lstm.shape[2])

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, shuffle=False)

# 🔹 8️⃣ Crear el modelo LSTM
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(look_back, X_train.shape[2])),  # Definir la entrada aquí
    tf.keras.layers.LSTM(units=50, return_sequences=True),
    tf.keras.layers.LSTM(units=50),
    tf.keras.layers.Dense(units=1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# 🔹 9️⃣ Entrenar el modelo
model.fit(X_train, y_train, epochs=20, batch_size=32)

# 🔹 🔟 Predecir el precio de apertura de AAPL en 3 días
predicted_price_scaled = model.predict(X_test[-1].reshape(1, look_back, X_test.shape[2]))

# Invertir la normalización de la predicción
predicted_price = scaler_y.inverse_transform(predicted_price_scaled)
print("............. Ultimos precios............")
print(df.tail(5))
print(f"Predicción del precio de apertura para AAPL en 3 días: ${predicted_price[0][0]:.2f}")

Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - loss: 0.0525
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0056
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0035
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.0031
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.0027
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0026
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0025
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0024
Epoch 9/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.0020
Epoch 10/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0025
Epoch 11/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 0.0019
Epoch 12/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 0.0021
Epoch 13/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 0.0019
Epoch 14/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0020
Epoch 15/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0020
Epoc